In [1]:
import os
localfs_path = os.environ.get('SCRATCH_LOCAL') + '/'

In [2]:
os.environ['_JAVA_OPTIONS'] = f'-Djava.io.tmpdir={localfs_path}'

import hail as hl

hl.init(
    tmp_dir=(localfs_path+'tmp_hail_3'),
    spark_conf={'spark.driver.memory': '10G', 'spark.executor.memory': '10G'},
    default_reference='GRCh38'
) 

Picked up _JAVA_OPTIONS: -Djava.io.tmpdir=/localfs/4665854/
Picked up _JAVA_OPTIONS: -Djava.io.tmpdir=/localfs/4665854/


23/09/08 16:26:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.3.2
SparkUI available at http://ac0437:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.113-cf32652c5077
LOGGING: writing to /net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/preprocessing/oligogenic-model/hail-20230908-1625-0.2.113-cf32652c5077.log


In [24]:
from hail.plot import show
from pprint import pprint
from bokeh.layouts import gridplot
import numpy as np
hl.plot.output_notebook()
import bokeh
from scipy import stats

from bokeh.plotting import figure, output_notebook, show
output_notebook

Loading BokehJS ...

<function bokeh.io.output.output_notebook(resources=None, verbose=False, hide_banner=False, load_timeout=5000, notebook_type='jupyter')>

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
from oligogenic_model import (
    read_in_pheno_anno_all
)

In [ ]:
mt = read_in_pheno_anno_all(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/gts_and_s.mt',
    'all')

2023-07-26 21:08:21.501 Hail: INFO: Reading table without type imputation
  Loading field 'f0' as type str (not specified)
  Loading field 'f1' as type str (not specified)
  Loading field 'f2' as type str (not specified)
  Loading field 'f3' as type str (not specified)
  Loading field 'f4' as type str (not specified)
  Loading field 'f5' as type str (not specified)
  Loading field 'f6' as type str (not specified)
  Loading field 'f7' as type str (not specified)
  Loading field 'f8' as type str (not specified)
2023-07-26 21:08:22.513 Hail: INFO: Reading table without type imputation
  Loading field 'Gene stable ID' as type str (not specified)
  Loading field 'UniProtKB Gene Name symbol' as type str (not specified)
2023-07-26 21:08:23.807 Hail: INFO: Reading table to impute column types
2023-07-26 21:08:24.820 Hail: INFO: Finished type imputation
  Loading field 'ID' as type str (imputed)
  Loading field 'family' as type str (imputed)
  Loading field 'sex' as type str (imputed)
  Loading

In [9]:
mt = hl.read_matrix_table('/localfs/4665854/all.mt')

In [10]:
samples_to_exclude = ['S_7288', 'S_7289', 'S_7290', 'WGS_6827', 'WGS_6835', 'B502', 'B506']

In [11]:
mt = mt.filter_cols(
    hl.literal(samples_to_exclude).contains(mt.s),
    keep = False
)

In [12]:
mt.count()

(6538550, 280)

In [13]:
mt.aggregate_cols(hl.agg.counter(mt.group))

{'GTS': 180, 'local_controls': 100}

In [15]:
mt = mt.filter_rows(hl.agg.any(mt.GT.is_non_ref()))

In [16]:
mt = mt.naive_coalesce(500)

In [ ]:
mt.write(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/for-qc-all.mt',
    overwrite = True
)

In [ ]:
mt = hl.read_matrix_table('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/for-qc-all.mt')

In [54]:
mt.aggregate_rows(
    hl.agg.counter(
        mt.vep.vep.most_severe_consequence
    )
)

{'3_prime_UTR_variant': 332008,
 '5_prime_UTR_variant': 95013,
 'TF_binding_site_variant': 7945,
 'coding_sequence_variant': 17,
 'downstream_gene_variant': 224909,
 'incomplete_terminal_codon_variant': 27,
 'intergenic_variant': 273782,
 'intron_variant': 4355912,
 'mature_miRNA_variant': 363,
 'missense_variant': 236333,
 'non_coding_transcript_exon_variant': 269293,
 'regulatory_region_variant': 104832,
 'splice_acceptor_variant': 1975,
 'splice_donor_variant': 3094,
 'splice_region_variant': 30457,
 'start_lost': 587,
 'stop_gained': 6538,
 'stop_lost': 903,
 'stop_retained_variant': 261,
 'synonymous_variant': 251561,
 'upstream_gene_variant': 318946}

In [10]:
mt.count()

(6514756, 280)

In [ ]:
mt = hl.variant_qc(mt)
mt = hl.sample_qc(mt)
mt = mt.checkpoint(localfs_path+'mt-qced.mt')

In [ ]:
mt = hl.read_matrix_table('/localfs/4665854/mt-qced.mt')

In [67]:
p = hl.plot.histogram(mt.sample_qc.call_rate, range=(.98,1), legend='Call Rate')
show(p)

In [27]:
p2 = hl.plot.histogram(mt.variant_qc.dp_stats.mean, range=(0,120), legend='Coverage'
                      )
show(p2)

In [24]:
mt.aggregate_cols(hl.agg.stats(mt.sample_qc.gq_stats.mean))

Struct(mean=75.93978638203325, stdev=13.583481489040917, min=13.385729516374342, max=97.20849875610891, n=280, sum=21263.14018696931)

In [55]:
mt.aggregate_cols(hl.agg.stats(mt.sample_qc.dp_stats.mean))

Struct(mean=29.730013283546572, stdev=7.360488628074394, min=6.627429804620724, max=57.15210555826873, n=280, sum=8324.40371939304)

In [59]:
mt.aggregate_rows(hl.agg.stats(mt.variant_qc.gq_stats.mean))

Struct(mean=75.93623586649194, stdev=6.966540687133007, min=40.36071428571429, max=99.0, n=6514756, sum=494706048.2286436)

In [61]:
mt.aggregate_rows(hl.agg.stats(mt.variant_qc.dp_stats.mean))

Struct(mean=29.728882857576213, stdev=20.614663891685222, min=11.767857142857142, max=3693.1035714285713, n=6514756, sum=193676417.96969178)